In [ ]:
try:
    %tensorflow_version 2.x
except:
    pass

In [ ]:
import tensorflow as tf
import numpy as np

print("\u2022 Using TensorFlow Version:", tf.__version__)

In [ ]:
!pip install -q requests

In [ ]:
import os
import json
import tempfile
import requests

In [ ]:
!echo "deb http://storage.googleapis.com/tensorflow-serving-apt stable tensorflow-model-server tensorflow-model-server-universal" | tee /etc/apt/sources.list.d/tensorflow-serving.list && \
curl https://storage.googleapis.com/tensorflow-serving-apt/tensorflow-serving.release.pub.gpg | apt-key add -
!apt update

In [ ]:
!apt-get install tensorflow-model-server

In [ ]:
MODEL_DIR = tempfile.gettempdir()

version = 1

export_path = os.path.join(MODEL_DIR, str(version))

!saved_model_cli show --dir {export_path} --all

In [ ]:
import sys
# We need sudo prefix if not on a Google Colab.
if 'google.colab' not in sys.modules:
  SUDO_IF_NEEDED = 'sudo'
else:
  SUDO_IF_NEEDED = ''

!{SUDO_IF_NEEDED} apt update

In [ ]:
!{SUDO_IF_NEEDED} apt-get install tensorflow-model-server

In [ ]:
os.environ["MODEL_DIR"] = MODEL_DIR

In [ ]:
%%bash --bg 
nohup tensorflow_model_server \
  --rest_api_port=8501 \
  --model_name=model \
  --model_base_path="${MODEL_DIR}" >server.log 2>&1

In [ ]:
!tail server.log

In [ ]:
X_predict = np.array([[210, 5.9]])
data = json.dumps({"signature_name": "serving_default", "instances": X_predict.tolist()})
print(data)

In [ ]:
labels = [1, 2, 3, 4, 5, 6, 7, 8, 9]
headers = {"content-type": "application/json"}
json_response = requests.post('http://localhost:8501/v1/models/model:predict', data=data, headers=headers)
predictions = json.loads(json_response.text)['predictions']

print('The condition is {}'.format(labels[np.argmax(predictions[0])]))